In [4]:
# ======================== IMPORTS ========================
from collections import Counter
from joblib import dump
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import VotingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVC
from sklearn.svm import SVR
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pycountry
import re
import seaborn as sns


In [5]:

# ======================== CLASSES ========================
class CorrelationFilter:

    def __init__(self, threshold=0.85):
        self.threshold = threshold
        self.to_drop = set()

    def fit(self, data):
        numerical_data = data.select_dtypes(include=[np.number])
        corr_matrix = numerical_data.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > self.threshold:
                    colname = corr_matrix.columns[i]
                    self.to_drop.add(colname)

    def transform(self, data):
        return data.drop(columns=list(self.to_drop & set(data.columns)), errors='ignore')

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)
class CategoryReducer:

    def __init__(self, category_columns, top_n=15):
        self.category_columns = category_columns
        self.top_n = top_n
        self.top_categories = None

    def fit(self, data):
        self.top_categories = data[self.category_columns].sum().sort_values(ascending=False).head(self.top_n).index.tolist()

    def transform(self, data):
        df_top = data[self.top_categories].copy()
        other_columns = list(set(self.category_columns) - set(self.top_categories))
        df_top['Other'] = data[other_columns].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
        data = data.drop(columns=self.category_columns)
        data = pd.concat([data, df_top], axis=1)
        return data

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)
class AgeTransformer:

    def __init__(self, current_year=2025):
        self.current_year = current_year
        self.age_mode = None

    def fit(self, data):
        data = data.copy()
        data['Year Founded'] = pd.to_numeric(data['Year Founded'], errors='coerce')
        data['age'] = self.current_year - data['Year Founded']
        mode_series = data['age'].mode()
        if not mode_series.empty:
            self.age_mode = mode_series[0]
        else:
            self.age_mode = 5

    def transform(self, data):
        """Applies the transformation: computes age and fills missing values."""
        data = data.copy()
        data['Year Founded'] = pd.to_numeric(data['Year Founded'], errors='coerce')
        data['age'] = self.current_year - data['Year Founded']
        data.drop(columns=['Year Founded'], inplace=True)
        data['age'].fillna(self.age_mode, inplace=True)
        return data
class IPOAgeTransformer:

    def __init__(self, current_year=2025):
        self.current_year = current_year

    def fit(self, data):
        return self

    def transform(self, data):
        df = data.copy()
        df['IPO'] = pd.to_numeric(df['IPO'], errors='coerce')
        df['age IPO'] = self.current_year - df['IPO']
        df['age IPO'].replace(np.nan, -1, inplace=True)
        if 'IPO' in df.columns:
            df.drop(columns=['IPO'], inplace=True)
        return df

    def fit_transform(self, data):
        return self.fit(data).transform(data)
class EmployeeDataCleaner:

    def __init__(self):
        self.employee_mode = None
        self.mean_without_zeros = None

    def fit(self, data):
        self.employee_mode = data['Number of Employees (year of last update)'].mode()[0] if not data['Number of Employees (year of last update)'].mode().empty else 0
        self.mean_without_zeros = data.loc[data['Number of Employees'] > 0, 'Number of Employees'].mean()
        return self

    def transform(self, data):
        df = data.copy()
        if 'Number of Employees (year of last update)' in df.columns:
            df['Number of Employees (year of last update)'].fillna(self.employee_mode, inplace=True)
        if 'Number of Employees' in df.columns:
            df['Number of Employees'] = df['Number of Employees'].apply(lambda x: 0 if pd.isna(x) or x < 0 else x)
            df['Number of Employees'] = df['Number of Employees'].replace(0, self.mean_without_zeros)
        return df

    def fit_transform(self, data):
        return self.fit(data).transform(data)
class TaglineCategoryGuesser:

    def __init__(self):
        self.category_keywords = {'Artificial Intelligence': ['ai', 'machine learning', 'deep learning', 'neural network'], 'Mobile': ['mobile', 'android', 'ios', 'app store', 'smartphone'], 'E-Commerce': ['ecommerce', 'e-commerce', 'shopping', 'online store'], 'FinTech': ['finance', 'banking', 'payments', 'fintech', 'crypto', 'blockchain'], 'Healthcare': ['health', 'medical', 'hospital', 'doctor', 'pharma'], 'Social Media': ['social network', 'community', 'messaging', 'chat'], 'Gaming': ['game', 'gaming', 'video game', 'esports'], 'Cloud': ['cloud', 'saas', 'paas', 'infrastructure'], 'EdTech': ['education', 'learning', 'students', 'teaching', 'school'], 'Data Analytics': ['analytics', 'data science', 'big data', 'insights']}

    def guess_category_from_tagline(self, tagline):
        tagline = str(tagline).lower()
        matched = [cat for (cat, keywords) in self.category_keywords.items() if any((keyword in tagline for keyword in keywords))]
        if len(matched) == 0:
            matched = ['Software', 'Advertising']
        elif len(matched) == 1:
            matched.append('Software')
        return ', '.join(matched)

    def fit(self, data):
        return self

    def transform(self, data):
        df = data.copy()
        df['Tagline'] = df['Tagline'].fillna('')
        df['Market Categories'] = df['Market Categories'].fillna('Unknown')
        df['Market Categories'] = df.apply(lambda row: self.guess_category_from_tagline(row['Tagline']) if str(row['Market Categories']).strip().lower() in ['unknown', 'nan', 'none', ''] else row['Market Categories'], axis=1)
        return df

    def fit_transform(self, data):
        return self.fit(data).transform(data)
class MarketCategoryGeneralizer:

    def __init__(self):
        self.category_mapping = {'Software': 'Technology & Software', 'Advertising': 'Advertising & Marketing', 'E-Commerce': 'E-Commerce & Online Services', 'Mobile': 'Mobile & Consumer Electronics', 'Games': 'Games & Entertainment', 'Social Media': 'Social Networking & Communication', 'Cloud': 'Technology & Software', 'Finance': 'Finance & Payments', 'Healthcare': 'Healthcare & Wellness', 'Semiconductors': 'Technology Hardware', 'Data Analytics': 'Analytics & Data Science', 'Search': 'Advertising & Marketing', 'Video': 'Games & Entertainment', 'Networking': 'Telecom & Networks', 'Messaging': 'Social Networking & Communication', 'Education': 'Education & Learning', 'News': 'Media & News', 'Photo Sharing': 'Digital Media & Content', 'Mobile Payments': 'Finance & Payments', 'Robotics': 'Games & Entertainment', 'Music': 'Games & Entertainment', 'Photo Editing': 'Digital Media & Content', 'Online Rental': 'E-Commerce & Online Services', 'Location Based Services': 'Telecom & Networks', 'Enterprise Software': 'Technology & Software', 'Video Streaming': 'Games & Entertainment', 'PaaS': 'Technology & Software', 'SaaS': 'Technology & Software', 'Health and Wellness': 'Healthcare & Wellness', 'Web Hosting': 'Technology & Software', 'Internet of Things': 'IoT (Internet of Things)', 'Cloud Security': 'Technology & Software', 'Virtual Currency': 'Finance & Payments', 'Search Marketing': 'Advertising & Marketing', 'Mobile Social': 'Social Networking & Communication', 'Retail': 'Retail & Fashion', 'Consulting': 'Others & Miscellaneous', 'Aerospace': 'Others & Miscellaneous', 'Food Delivery': 'Consumer Goods & Services', 'Fashion': 'Retail & Fashion', 'Wine And Spirits': 'Consumer Goods & Services', 'Streaming': 'Games & Entertainment', 'Task Management': 'Others & Miscellaneous', 'Video Chat': 'Social Networking & Communication', 'Personalization': 'Advertising & Marketing', 'Shopping': 'E-Commerce & Online Services', 'Local': 'E-Commerce & Online Services', 'News': 'Media & News', 'Fraud Detection': 'Advertising & Marketing', 'Image Recognition': 'Technology Hardware', 'Virtualization': 'Games & Entertainment', 'Analytics': 'Analytics & Data Science', 'Video on Demand': 'Games & Entertainment', 'Mobile Payments': 'Finance & Payments', 'Marketing Automation': 'Advertising & Marketing', 'Consumer Electronics': 'Mobile & Consumer Electronics', 'Video Games': 'Games & Entertainment', 'Public Relations': 'Advertising & Marketing'}

    def map_categories(self, row):
        categories = str(row).split(',')
        generalized = []
        for cat in categories:
            cat = cat.strip()
            if cat in self.category_mapping:
                generalized.append(self.category_mapping[cat])
            else:
                generalized.append('Others & Miscellaneous')
        return ', '.join(set(generalized))

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        df['Generalized Market Categories'] = df['Market Categories'].fillna('').apply(self.map_categories)
        return df

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
class CountryRegionFiller:

    def __init__(self):
        self.countries = [country.name for country in pycountry.countries]
        self.regions = ['California', 'New York', 'Texas', 'Basel', 'Utah', 'Île-de-France', 'Bavaria', 'Ontario', 'Switzerland', 'United States', 'France', 'Great Britain', 'Israel', 'Sweden', 'Canada', 'Germany', 'Japan', 'India', 'Denmark', 'China', 'Spain', 'Netherlands', 'Finland', 'Australia', 'Ireland', 'United Stats of AMerica', 'United Arab Emirates', 'Quebec']

    def find_place(self, text, place_list):
        for place in place_list:
            if re.search('\\b' + re.escape(place) + '\\b', str(text)):
                return place
        return None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        for (idx, row) in df[df['Country (HQ)'].isnull() | df['State / Region (HQ)'].isnull()].iterrows():
            desc = row['Description']
            if pd.isnull(desc):
                continue
            country = self.find_place(desc, self.countries)
            region = self.find_place(desc, self.regions)
            if pd.isnull(row['Country (HQ)']) and country:
                df.at[idx, 'Country (HQ)'] = country
            if pd.isnull(row['State / Region (HQ)']) and region:
                df.at[idx, 'State / Region (HQ)'] = region
        return df

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)
class CategoricalFillerAndEncoder:

    def __init__(self, columns):
        self.columns = columns
        self.modes = {}
        self.label_encoders = {}
        self.label_maps = {}

    def fit(self, X, y=None):
        for col in self.columns:
            mode_val = X[col].mode()[0]
            self.modes[col] = mode_val
            le = LabelEncoder()
            filled = X[col].fillna(mode_val).astype(str)
            le.fit(filled)
            self.label_encoders[col] = le
            self.label_maps[col] = {label: i for (i, label) in enumerate(le.classes_)}
        return self

    def transform(self, X):
        df = X.copy()
        for col in self.columns:
            mode_val = self.modes[col]
            label_map = self.label_maps[col]
            df[col] = df[col].fillna(mode_val).astype(str)
            df[col + '_LabelEncoded'] = df[col].map(lambda x: label_map.get(x, -1))
        return df

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
class CustomEncoder(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.status_cols_ = None
        self.terms_cols_ = None

    def fit(self, df):
        self.status_cols_ = df['Status'].unique()
        self.terms_cols_ = df['Terms'].unique()
        return self

    def transform(self, df):
        df = df.copy()
        df = pd.get_dummies(df, columns=['Status'], drop_first=False)
        df = pd.get_dummies(df, columns=['Terms'], drop_first=False)
        if 'Terms_Cash, Stock' in df.columns:
            cash_stock_mask = df['Terms_Cash, Stock'] == 1
            df.loc[cash_stock_mask, 'Terms_Cash'] = 1
            df.loc[cash_stock_mask, 'Terms_Stock'] = 1
            df = df.drop('Terms_Cash, Stock', axis=1)
        expected_cols = [f'Status_{s}' for s in self.status_cols_] + [f'Terms_{t}' for t in self.terms_cols_ if t != 'Cash, Stock']
        for col in expected_cols:
            if col not in df.columns:
                df[col] = 0
        return df[expected_cols]

In [6]:


def predict_new_data(acquiring_path, acquired_path, acquisitions_path, output_path):
    """
    Process new data and make predictions using saved models
    
    Args:
        acquiring_path: Path to new acquiring companies CSV
        acquired_path: Path to new acquired companies CSV  
        acquisitions_path: Path to new acquisitions CSV
        output_path: Where to save predictions
    """
    
    # ===================================
    # Load All Saved Preprocessing Objects
    # ===================================
    
    # Load acquiring company transformers
    with open('mlb_acquiring.pkl', 'rb') as f:
        mlb_acquiring = pickle.load(f)
    
    with open("correlation_filte_acquiring.pkl", "rb") as f:
        loaded_filter = pickle.load(f)



    #
    with open("category_reducer_acquiring.pkl", "rb") as f:
        loaded_reducer = pickle.load(f)

    #
    with open('Age_column_acquiring.pkl', 'rb') as f:
        age_mode_col =  pickle.load(f)
    #
    with open("ipo_transformer_acquiring.pkl", "rb") as f:
        loaded_ipo_transformer = pickle.load(f)
    #
    with open("employee_cleaner_acquiring.pkl", "rb") as f:
        loaded_employee_cleaner = pickle.load(f)


    with open('tfidf_acquiring.pkl', 'rb') as f:
        tfidf_acquiring = pickle.load(f)
        
    #
    with open("tagline_guesser_acquired.pkl", "rb") as f:
        loaded_guesser = pickle.load(f)

    #
    with open("category_generalizer_acquired.pkl", "rb") as f:
        loaded_generalizer = pickle.load(f)

    #
    with open("country_region_filler_acquired.pkl", "rb") as f:
        loaded_filler = pickle.load(f)
    
    #
    with open("categorical_encoder_acquired.pkl", "rb") as f:
        encoder = pickle.load(f)

    # Load acquired company transformers
    with open('mlb_acquired.pkl', 'rb') as f:
        mlb_acquired = pickle.load(f)
    # with open('label_encoders_acquired.pkl', 'rb') as f:
    #     label_encoders_acquired = pickle.load(f)
        
    # Load acquisitions transformers
    # with open('ohe_acquisitions.pkl', 'rb') as f:
    #     ohe_acquisitions = pickle.load(f)
        
    # Load final preprocessing objects
    # with open('final_imputer.pkl', 'rb') as f:
    #     final_imputer = pickle.load(f)
    with open('final_scaler.pkl', 'rb') as f:
        final_scaler = pickle.load(f)
    with open('final_pca.pkl', 'rb') as f:
        final_pca = pickle.load(f)
    with open('target_encoder.pkl', 'rb') as f:
        target_encoder = pickle.load(f)
    

# Load saved preprocessing objects
    
    # Load model
    # with open('final_model.pkl', 'rb') as f:
    #     model = pickle.load(f)
    
    # ==============================
    # Preprocess New Data (Same as Training)
    # ==============================
    
    # Process each dataset with saved transformers
    def process_acquiring_new(data):
        """Process new acquiring data with saved transformers"""
        data = data.copy()
        
        # Apply same cleaning as training
        data.drop(['CrunchBase Profile','Image','Homepage','Twitter','API'], 
                 axis=1, inplace=True, errors='ignore')
                
        data['Number of Employees'] = data['Number of Employees'].replace({',': ''}, regex=True)
        data['Number of Employees'] = data['Number of Employees'].fillna(0).astype(int)
        
        # Use mean from training (would need to save this)
        data['Number of Employees'] = data['Number of Employees'].replace(
            0, 450)  # Replace with saved mean from training
        
        # Handle IPO status
        data['IPO'] = data['IPO'].replace("Not yet", np.nan)
        data['Is_Public'] = data['IPO'].notna().astype(int)
        data.drop('IPO', axis=1, inplace=True)

        # Process Market Categories with saved MLB
        data['Market Categories'] = data['Market Categories'].fillna('')
        category_dummies = pd.DataFrame(
            mlb_acquiring.transform(data['Market Categories'].str.split(',')),
            columns=mlb_acquiring.classes_,
            index=data.index
        )
        
        #
        data = loaded_filter.transform(data)


        # Use the loaded reducer on new data
        data = loaded_reducer.transform(data)

        #
        data = age_mode_col.transform(data)

        #
        data = loaded_ipo_transformer.transform(data)

        #
        data = loaded_employee_cleaner.transform(data)



        # Process text with saved TF-IDF
        data['Text_Combined'] = data['Tagline'].fillna('') + ' ' + data['Description'].fillna('')
        def clean_text(text):
            text = text.lower()
            text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation/numbers
            text = re.sub(r'\s+', ' ', text).strip()  # remove extra whitespace
            return text

        data['Text_Combined'] = data['Text_Combined'].apply(clean_text)
        tfidf_features = tfidf_acquiring.transform(data['Text_Combined'])
        tfidf_df = pd.DataFrame(
            tfidf_features.toarray(), 
            columns=tfidf_acquiring.get_feature_names_out(),
            index=data.index
        )
        
        # Final cleanup
        data = pd.concat([data, category_dummies, tfidf_df], axis=1)
        data.drop(['Market Categories', 'Tagline', 'Description', 'Text_Combined','Address (HQ)',
                   'Board Members','Founders'], 
                 axis=1, inplace=True)
        
        return data
    
    # Similar functions for acquired and acquisitions...
    # (Implementation would mirror the training preprocessing but using saved transformers)
    

    def process_acquisitions_new(data):
        """Process new acquisitions data with saved transformers"""
        data = data.copy()
        
        
        # Drop columns same as training
        data.drop(columns=["Acquisition Profile", "News", "News Link"], 
                inplace=True, errors='ignore')
        
        with open('custom_acquisitions_encoder.pkl', 'rb') as f:
            encoder = pickle.load(f)
        with open('mode_acquisitions_imputer.pkl', 'rb') as f:
            modes = pickle.load(f)

        # Handle missing values with saved modes
        for col, mode_val in modes.items():
            if col in data.columns and mode_val is not None:
                data[col].fillna(mode_val, inplace=True)
        
        # Convert date features same as training
        if 'Deal announced on' in data.columns:
            data['Deal_date'] = pd.to_datetime(data['Deal announced on'], dayfirst=True, errors='coerce')
            data['Deal_day'] = data['Deal_date'].dt.day
            data['Deal_month'] = data['Deal_date'].dt.month
            data['Deal_dayofweek'] = data['Deal_date'].dt.dayofweek
            data.drop(['Deal announced on', 'Deal_date'], axis=1, inplace=True)
        
        # Apply saved one-hot encoding
        encoded_status_terms = encoder.transform(data)
        data = pd.concat([data, encoded_status_terms], axis=1)
        
        return data

    def process_acquired_new(data):
        """Process new acquiring data with saved transformers"""
        data = data.copy()

        data = loaded_guesser.transform(data)

        data = loaded_generalizer.transform(data)

        data = loaded_filler.transform(data)

        data = encoder.transform(data)
        
        data['Acquired by'].fillna('Salesforce', inplace=True)


        columns_to_drop = ['Image', 'CrunchBase Profile', 'Homepage', 'Twitter','Address (HQ)','API','Description',
                           'Tagline','Market Categories','City (HQ)', 'State / Region (HQ)', 'Country (HQ)','Generalized Market Categories','Year Founded']
        data.drop(columns=columns_to_drop, inplace=True)
        
        return data 
    
    
    
    # Preprocess each new dataset
    acquiring_new = process_acquiring_new(pd.read_csv(acquiring_path))
    acquired_new = process_acquired_new(pd.read_csv(acquired_path)) 
    acquisitions_new = process_acquisitions_new(pd.read_csv(acquisitions_path))
    
    # Merge the new data (same as training)
    final_new = merge_datasets(
        acquiring_new, acquired_new, acquisitions_new, save_artifacts=False
    )
    
    # Handle missing values with saved imputer
    final_new_imputed = pd.DataFrame(
        final_imputer.transform(final_new),
        columns=final_new.columns
    )
    
    # Prepare features
    X_new = final_new_imputed.drop(
        ['Deal size class', 'Acquired Company', 'Acquiring Company'], 
        axis=1, errors='ignore'
    )
    
    # Apply same scaling and PCA as training
    X_new_scaled = final_scaler.transform(X_new)
    X_new_pca = final_pca.transform(X_new_scaled)
    
    # Make predictions
    predictions_encoded = model.predict(X_new_pca)
    predictions = target_encoder.inverse_transform(predictions_encoded)
    
    # Save predictions with original data
    final_new['Predicted_Deal_Size'] = predictions
    final_new.to_csv(output_path, index=False)
    
    print(f"Predictions saved to {output_path}")

if __name__ == "__main__":
    predict_new_data(
        acquiring_path="new_acquiring.csv",
        acquired_path="new_acquired.csv",
        acquisitions_path="new_acquisitions.csv",
        output_path="new_predictions.csv"
    )

/home/kero/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [' Advertising Platforms', ' All Markets', ' All Students', ' Auctions', ' Blogging Platforms', ' Cloud Computing', ' Colleges', ' Communities', ' Consulting', ' Consumer Electronics', ' Consumer Goods', ' Content Creators', ' Creative', ' Crowdsourcing', ' Curated Web', ' Design', ' Digital Media', ' E-Commerce', ' Email', ' Enterprise Software', ' Enterprises', ' Facebook Applications', ' Hardware', ' Hardware + Software', ' Identity', ' Image Recognition', ' Information Technology', ' MicroBlogging', ' Mobile', ' Networking', ' Photography', ' RIM', ' SMS', ' Search', ' Security', ' Social Bookmarking', ' Social Media', ' Software', ' Storage', ' Technology', ' Video Streaming', ' Web Hosting', ' Wireless'] will be ignored
  warnings.warn(


KeyError: "None of [Index(['Software', 'Hardware + Software', 'Mobile', 'Social Media',\n       'Curated Web', 'Web Hosting', 'Search', 'Networking',\n       'Enterprise Software', 'Email', 'Security', 'Hardware',\n       'Information Technology', 'E-Commerce', 'Electronics'],\n      dtype='object')] are in the [columns]"

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox

def browse_file(entry_widget):
    directory = filedialog.askopenfilename()
    if directory:
        entry_widget.delete(0, tk.END)
        entry_widget.insert(0, directory)

def predict_data():
    # Get input values from entries
    paths = [
        entry1.get(),
        entry2.get(),
        entry3.get()
    ]
    
    # Validate paths
    for path in paths:
        if not path.strip():
            messagebox.showerror("Error", "All paths must be selected!")
            return
    
    # Call your prediction function
    try:
        predict_new_data(*paths, "new_predictions.csv")
        messagebox.showinfo("Success", "Prediction completed successfully!")
    except Exception as e:
        messagebox.showerror("Error", f"Prediction failed: {str(e)}")

# Create main window
root = tk.Tk()
root.title("Predictor")
root.geometry("1200x800")

# Function to create consistent input fields with browse buttons
def create_file_input(row, label_text):
    # Label
    tk.Label(root, text=label_text).grid(row=row, column=0, padx=10, pady=5, sticky=tk.W)
    
    # Entry field
    entry = tk.Entry(root, width=80)
    entry.grid(row=row, column=1, padx=10, pady=5)
    
    # Browse button
    browse_btn = tk.Button(root, text="Browse", 
                          command=lambda: browse_file(entry))
    browse_btn.grid(row=row, column=2, padx=10, pady=5)
    
    return entry

# Create input fields with browse buttons
entry1 = create_file_input(0, "Acquiring Path:")
entry2 = create_file_input(1, "Acquired Path:")
entry3 = create_file_input(2, "Acquisitions Path:")

# Prediction button
predict_button = tk.Button(root, text="Predict", command=predict_data,
                          bg="#4CAF50", fg="white", font=("Arial", 12))
predict_button.grid(row=3, column=0, columnspan=3, pady=20, ipadx=10, ipady=5)

# Configure grid layout
root.grid_columnconfigure(1, weight=1)  # Make entry fields expandable

# Start the GUI
root.mainloop()